# Installation and Setup

In [1]:
!pip install gensim sentence-transformers

In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Download and Import Dataset

Pada penugasan kali ini, kita menggunakan 2 dataset untuk membangun sistem rekomendasi, yaitu:

### 1. Job Title and Job Description Dataset
Dataset ini berisi koleksi judul pekerjaan beserta deskripsi lengkapnya yang mencakup tanggung jawab dan kualifikasi.

Kolom dataset:
- `Posisi`: Judul pekerjaan
- `Perusahaan`: Nama perusahaan yang membuka lowongan.
-	`Lokasi`: Lokasi penempatan kerja.
-	`Gaji`:	Informasi mengenai kisaran atau detail gaji (jika tersedia).
- `Waktu`: Kapan lowongan tersebut diunggah atau diperbarui.
- `Deskripsi_Stem`: Ringkasan singkat mengenai tanggung jawab dan kualifikasi pekerjaan.

### 2. Coursera Courses Dataset 2024
Dataset ini memuat informasi kursus yang tersedia di platform Coursera, termasuk nama kursus, universitas penyelenggara, tingkat kesulitan, rating, link ke kursus, deskripsi kursus, dan skill yang diajarkan.

Kolom dataset:
- `Title`: Judul nama kursus.
- `Organization`: Nama organisasi atau partner industri yang menawarkan kursus.
- `Skills`: Kumpulan skill yang diajarkan dalam kursus
- `Ratings`: Rating kursus pada skala 5 poin
- `course_url`: Link langsung ke halaman kursus
- `course_students_enrolled`: Jumlah total peserta yang telah mendaftar dalam kursus
- `course_description`: Deskripsi detail isi kursus
- `Review Count`: Jumlah ulasan yang telah diberikan oleh peserta
- `Difficulty`: Tingkat kesulitan kursus
- `Type`: Jenis kursus, misalnya Professional Certificate, Specialization, Course, dll.
- `Duration`: Perkiraan waktu yang dibutuhkan untuk menyelesaikan kursus, biasanya dalam rentang bulan atau minggu (contoh: 3 - 6 Months).

Sumber Dataset:
- Job Title and Job Description Dataset = penugasan pada Hands On 1
- [Coursera Courses Dataset 2024](https://www.kaggle.com/datasets/azraimohamad/coursera-course-data?resource=download&select=coursera_course_dataset_v3.csv)

In [4]:
df_job = pd.read_csv('/content/Full_Data_Job_Listings_Clean.csv')
df_job.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi_Stem
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,look data scientist equal comfort write sql qu...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,career growth opportun rapid grow compani
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,minimum 5 year experi data scientist strong pr...
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,use data understand user need defacto voic cus...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,budaya kerja kekeluargaan


In [5]:
df_course = pd.read_csv('/content/coursera_course_dataset_v3.csv').drop('Unnamed: 0', axis=1)
df_course.head()

,Title,Organization,Skills,Ratings,course_url,course_students_enrolled,course_description,Review Count,Difficulty,Type,Duration
0,Google Cybersecurity,Google,"Network Security, Python Programming, Linux, ...",4.8,https://www.coursera.org/professional-certific...,"700,909",Google Cloud Fundamentals: Core Infrastructure...,20K,Beginner,Professional Certificate,3 - 6 Months
1,Google Data Analytics,Google,"Data Analysis, R Programming, SQL, Business C...",4.8,https://www.coursera.org/professional-certific...,"229,865",Prepare for a new career in the high-growth fi...,137K,Beginner,Professional Certificate,3 - 6 Months
2,Google Project Management:,Google,"Project Management, Strategy and Operations, ...",4.8,https://www.coursera.org/professional-certific...,"29,702",Prepare-se para uma nova carreira no campo de ...,100K,Beginner,Professional Certificate,3 - 6 Months
3,IBM Data Science,IBM,"Python Programming, Data Science, Machine Lea...",4.6,https://www.coursera.org/professional-certific...,"239,622",Prepare for a career in the high-growth field ...,120K,Beginner,Professional Certificate,3 - 6 Months
4,Google Digital Marketing & E-commerce,Google,"Digital Marketing, Marketing, Marketing Manag...",4.8,https://www.coursera.org/professional-certific...,"384,238",This course is the eighth course in the Google...,23K,Beginner,Professional Certificate,3 - 6 Months


## Text Processing

Untuk membangun sistem rekomendasi, kita menggunakan akan kolom `Job Description` dari dataset pekerjaan sebagai representasi teks pekerjaan. Sedangkan untuk dataset kursus, beberapa kolom yang relevan akan digabungkan menjadi satu kolom **`text`** yang mewakili informasi kursus secara menyeluruh, antara lain:

- `Title`
- `Difficulty`
- `course_description`
- `Skills`

Penggabungan ini bertujuan agar seluruh informasi relevan terwakili dalam satu kolom teks yang akan diproses dan digunakan dalam sistem rekomendasi.

In [6]:
df_course['text'] = df_course['Title'] +  ' ' + df_course['Difficulty'] + ' ' + df_course['course_description'] + ' ' + df_course['Skills']
df_course = df_course[['Title', 'text']]
df_course.head()

,Title,text
0,Google Cybersecurity,Google Cybersecurity Beginner Google Cloud Fun...
1,Google Data Analytics,Google Data Analytics Beginner Prepare for a n...
2,Google Project Management:,Google Project Management: Beginner Prepare-se...
3,IBM Data Science,IBM Data Science Beginner Prepare for a career...
4,Google Digital Marketing & E-commerce,Google Digital Marketing & E-commerce Beginner...


Kemudian, kita akan membersihkan serta memproses data teks. Ini termasuk mengubah menjadi huruf kecil, menghapus tag HTML, URL, hashtag, tanda baca, angka, dan spasi ekstra. Kita juga menghapus stop words dan menerapkan stemming untuk mengurangi kata-kata ke bentuk dasarnya.

Dengan pembersihan dan normalisasi ini, kita mendapatkan representasi teks yang bersih dan siap untuk divectorize serta digunakan dalam proses penghitungan similarity untuk rekomendasi.

In [8]:
# mengubah nilai NaN atau angka menjadi string kosong
df_course['text'] = df_course['text'].fillna('').astype(str)
df_job['Deskripsi_Stem'] = df_job['Deskripsi_Stem'].fillna('').astype(str)

In [9]:
def clean_noise(text):
  text = re.sub(r'<.*?>', ' ', text) # Hapus tag HTML
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text) # Hapus URL
  text = re.sub(r'#\w+', ' ', text) # Hapus hashtag
  text = re.sub(r'[^\w\s]', ' ', text) # Hapus tanda baca dan karakter khusus
  text = re.sub(r'\d+', ' ', text) # Hapus angka
  text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi berlebih
  return text

def remove_stopwords(text):
  stopwords_set = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stopwords_set]
  return ' '.join(filtered_words)

def stem_text(text):
  stemmer = PorterStemmer()
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  return ' '.join(stemmed_words)

def process(text):
    text = text.lower()
    text = clean_noise(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    return text

In [10]:
df_course['text'] = df_course['text'].apply(process)
df_course.head()

,Title,text
0,Google Cybersecurity,googl cybersecur beginn googl cloud fundament ...
1,Google Data Analytics,googl data analyt beginn prepar new career hig...
2,Google Project Management:,googl project manag beginn prepar se para uma ...
3,IBM Data Science,ibm data scienc beginn prepar career high grow...
4,Google Digital Marketing & E-commerce,googl digit market e commerc beginn cours eigh...


In [11]:
df_job['Deskripsi_Stem'] = df_job['Deskripsi_Stem'].apply(process)
df_job.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi_Stem
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,look data scientist equal comfort write sql qu...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,career growth opportun rapid grow compani
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,minimum year experi data scientist strong pred...
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,use data understand user need defacto voic cus...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,budaya kerja kekeluargaan


# 1. Data Vectorization

Sebelum kita dapat membuat sistem rekomendasi, langkah pertama yang harus dilakukan adalah dataset teks perlu diubah menjadi representasi numerik (vektor). Hal ini bertujuan agar kita bisa melihat kemiripan dari setiap kursus dan dapat melakukan analisis lebih lanjut.

## BERT/Sentence Transformers

Metode ini menggunakan model yang sudah dirancang untuk mengubah satu kalimat utuh menjadi suatu vektor yang merepresentasikan makna keseluruhan kalimat. Oleh karena itu, kita hanya tinggal menginisialisasi modelnya, lalu implementasikan ke dataset teks yang kita punya.

In [14]:
# Inisialisasi model pretrained BERT
model_bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text kursus menjadi vektor
course_vectors = model_bert.encode(df_course['text'])

# Menampilkan matriks fitur dalam bentuk array
print("\nMatriks BERT:")
print(course_vectors)

# Simpan vektor kursus
df_course_vectors = pd.DataFrame(course_vectors)
df_course_vectors.to_csv("bert_vectors_course.csv", index=False)


Matriks BERT:
[[-0.30937588 -0.27189192 -0.30137455 ... -0.72844833  0.26588488
   0.01418005]
 [-0.50251764 -0.06273969 -0.17699276 ... -0.60099095  0.10625932
   0.21066952]
 [-0.24198134  0.16650125 -0.25104895 ... -0.56094587  0.00114253
   0.09884392]
 ...
 [ 0.15472719  0.18004417  0.05891585 ... -0.09824565  0.4268204
   0.2178519 ]
 [ 0.15472713  0.18004431  0.05891575 ... -0.09824565  0.42682058
   0.21785185]
 [ 0.15472722  0.18004408  0.0589159  ... -0.09824541  0.42682055
   0.21785185]]


In [15]:
# Encode deskripsi pekerjaan menjadi vektor
job_vectors = model_bert.encode(df_job['Deskripsi_Stem'])

# Menampilkan matriks fitur dalam bentuk array
print("\nMatriks BERT:")
print(job_vectors)

# Simpan vektor pekerjaan
df_job_vectors = pd.DataFrame(job_vectors)
df_job_vectors.to_csv("bert_vectors_job.csv", index=False)


Matriks BERT:
[[-0.06619975  0.07481638 -0.38922778 ... -0.19015661 -0.11237016
   0.04165871]
 [-0.06105448  0.07181004 -0.12267227 ... -0.40717354  0.73416007
   0.35854077]
 [ 0.1539414  -0.37441847 -0.20325302 ... -0.17019016 -0.38027507
  -0.11227824]
 ...
 [-0.50363594  0.6475352  -0.54305965 ... -0.5319492   0.41813686
  -0.06257242]
 [ 0.41305616 -0.39588413 -0.05416235 ... -0.36277762 -0.6692584
   0.33637565]
 [-0.3624561  -0.33728004 -0.3738454  ...  0.24200784  0.02105901
   0.2021647 ]]


# 2. Similarity Calculation

Setelah kita mendapatkan representasi vektor untuk setiap teks dari dataset teks kita, kita dapat menghitung similaritas antara setiap teks pekerjaan dan setiap teks kursus. Di sini, kita menggunakan metode *Cosine Similarity* untuk mengukur tingkat kemiripan antar vektor teks tersebut.

In [17]:
# Hitung similaritas antara setiap vektor pekerjaan dengan setiap vektor kursus
similarity = cosine_similarity(job_vectors, course_vectors)

# Matrix hasil similaritas vector
print("Matrix Cosine Similarity:")
print(similarity)

# Simpan matriks cosine similarity
df_similarity = pd.DataFrame(similarity)

# (opsional) tambahkan nama posisi pekerjaan sebagai baris dan kursus sebagai kolom
df_similarity.index = df_job["Posisi"]  # pastikan kolom ini ada
df_similarity.columns = df_course["Title"]  # pastikan kolom ini ada
df_similarity.head()

df_similarity.to_csv("job_course_similarity_matrix.csv")

Matrix Cosine Similarity:
[[ 0.19890174  0.26353723  0.07956454 ... -0.02907962 -0.02907958
  -0.0290796 ]
 [ 0.16871315  0.36568692  0.31515688 ...  0.10987205  0.1098721
   0.10987206]
 [ 0.25842142  0.26352882  0.14939852 ...  0.02857177  0.02857178
   0.02857172]
 ...
 [ 0.36537194  0.4325465   0.19977453 ...  0.11542849  0.11542851
   0.11542846]
 [ 0.06513765  0.04305742  0.0782495  ...  0.15678431  0.15678427
   0.1567843 ]
 [ 0.27039093  0.46241027  0.26036    ...  0.0484316   0.04843169
   0.04843165]]


# 3. Build Recommender System

Setelah kita berhasil melakukan vektorisasi teks dan menghitung similarity, langkah selanjutnya adalah membangun sistem rekomendasi. Kita akan menggunakan content-based recommender system. Artinya, sistem akan merekomendasikan kursus berdasarkan kemiripan isi antara pekerjaan dan kursus. Kemiripan dihitung dari hasil vectorization dan similarity calculation di langkah sebelumnya.

Disini, kita mendefinisikan fungsi `recommend` yang mengambil nama pekerjaan sebagai input dan mengembalikan daftar kursus yang direkomendasikan berdasarkan cosine similarity yang telah dihitung. Fungsi ini menemukan indeks pekerjaan input, mendapatkan skor similarity pekerjaan tersebut dengan semua kursus yang tersedia, mengurutkannya dalam urutan menurun, dan mengembalikan nama-nama kursus teratas yang paling sesuai dengan pekerjaan tersebut.

In [18]:
def recommend(Posisi, top_n=5):
    # Cari indeks job pada DataFrame
    job_index = df_job[df_job['Posisi'] == Posisi].index[0]

    # Ambil skor similarity job ini dengan semua kursus
    job_similarities = similarity[job_index]

    # Dapatkan indeks top-N kursus dengan similarity tertinggi
    top_course_indices = job_similarities.argsort()[::-1][:top_n]

    # Ambil nama kursus rekomendasi
    recommended_courses = df_course.iloc[top_course_indices]['Title'].tolist()

    return recommended_courses

Mari kita uji sistem rekomendasi dengan contoh 5 pekerjaan pertama.

In [19]:
for job_name in df_job['Posisi'].tolist()[:5]:
    recommended_courses = recommend(job_name)

    print(f"Recommended courses for {job_name}:")
    for i, course_name in enumerate(recommended_courses, start=1):
        print(f"{i}. {course_name}")
    print()

Recommended courses for Data Scientist:
1. Deep Learning
2. Decentralized Finance (DeFi): The Future of Finance
3. Applied Software Engineering Fundamentals
4. Learn SQL Basics for Data Science
5. Python, Bash and SQL Essentials for Data Engineering

Recommended courses for Junior Data Scientist:
1. Epidemiology in Public Health Practice
2. IBM IT Scrum Master
3. Entrepreneurial Finance: Strategy and Innovation
4. IBM Back-End Development
5. Google IT Automation with Python

Recommended courses for Senior Data Scientist:
1. Applied Data Science with Python
2. Databases and SQL for Data Science with Python
3. What is Data Science?
4. Python and Statistics for Financial Analysis
5. Generative Adversarial Networks (GANs)

Recommended courses for Data Scientist (Analytics):
1. Meta Database Engineer
2. Data Literacy
3. Decentralized Finance (DeFi): The Future of Finance
4. Data Structures and Algorithms
5. Databases and SQL for Data Science with Python

Recommended courses for Data Analyst

# 4. Evaluation

Setelah membangun sistem rekomendasi, langkah selanjutnya adalah evaluasi untuk mengukur seberapa baik kinerja sistem yang telah dibuat. Evaluasi ini penting untuk memastikan bahwa rekomendasi yang diberikan benar-benar berguna dan relevan bagi pengguna. Evaluasi dapat dilakukan secara manual maupun kuantitatif, tergantung ketersediaan ground truth (label relevansi).

## 4.1. Manual Inspection

Manual Inspection adalah metode evaluasi yang paling sederhana yaitu dengan memeriksa secara langsung apakah hasil rekomendasi masuk akal dan relevan. Cara ini sangat cocok untuk tahap awal, terutama jika belum tersedia data relevansi (ground truth).

Evaluasi manual dapat dilakukan dengan:
- Membaca judul dan deskripsi rekomendasi kursus.
- Menilai secara subjektif apakah rekomendasi tersebut sudah sesuai dengan kebutuhan job.

In [20]:
for job_name in df_job['Posisi'].tolist()[2:3]:
    recommended_courses = recommend(job_name)

    print(f"Recommended courses for {job_name}:")
    for i, course_name in enumerate(recommended_courses, start=1):
        print(f"{i}. {course_name}")
    print()

Recommended courses for Senior Data Scientist:
1. Applied Data Science with Python
2. Databases and SQL for Data Science with Python
3. What is Data Science?
4. Python and Statistics for Financial Analysis
5. Generative Adversarial Networks (GANs)



Pada contoh di atas, sistem merekomendasikan beberapa kursus untuk posisi pekerjaan "Senior Data Scientist". Berdasarkan evaluasi manual, rekomendasi "Applied Data Science with Python", "Databases and SQL for Data Science with Python", serta "Python and Statistics for Financial Analysis" dinilai cukup relevan, karena berkaitan erat dengan kemampuan teknikal dan analitis yang dibutuhkan oleh seorang Data Scientist tingkat lanjut. Kursus "What is Data Science?" mungkin terlalu mendasar untuk level senior, sedangkan "Generative Adversarial Networks (GANs)" merupakan topik yang sangat spesifik dan lebih cocok untuk spesialis machine learning atau peneliti AI, bukan untuk generalis Data Scientist senior.

Evaluasi manual ini menunjukkan bahwa sistem sudah cukup baik dalam memilih kursus yang berorientasi teknis, namun perlu lebih cermat dalam menyesuaikan tingkat kedalaman materi dengan jenjang posisi pengguna. Hasil evaluasi semacam ini memberikan insight penting untuk mengembangkan sistem rekomendasi yang lebih cerdas dan adaptif terhadap konteks pengguna. Meski demikian, pendekatan manual ini tetap memiliki keterbatasan, seperti subjektivitas dalam penilaian, potensi inkonsistensi antar evaluator, serta ketidakefisienan saat menangani dataset yang besar.

Oleh karena itu, disarankan agar evaluasi manual dilengkapi dengan metrik evaluasi objektif dan kuantitatif, seperti precision\@k, recall\@k, dan mean average precision. Metrik-metrik ini memungkinkan evaluasi sistematis dan perbandingan yang adil antar algoritma rekomendasi. Pendekatan gabungan antara evaluasi manual untuk mendapatkan insight kualitatif dan metrik kuantitatif untuk pengukuran objektif akan membantu memberikan penilaian yang lebih menyeluruh dan meningkatkan kualitas sistem rekomendasi secara signifikan.


## 4.2. Evaluation Metrics

Jika terdapat ground truth berupa anotasi relevansi antara lowongan kerja dan kursus, maka evaluasi sistem rekomendasi dapat dilakukan secara kuantitatif dengan menggunakan berbagai metrik.

Contoh metrik yang dapat digunakan, yaitu:

1. Precision@k

    Precision@k mengukur proporsi rekomendasi yang relevan di antara k rekomendasi teratas. Nilai metrik ini mencerminkan seberapa akurat sistem dalam menampilkan kursus yang sesuai kebutuhan pengguna di posisi atas daftar rekomendasi.

$$
\text{Precision@k} = \frac{\text{Jumlah kursus relevan pada top-}k}{k}
$$

2. Recall@k

    Recall@k menghitung persentase item relevan yang berhasil direkomendasikan dalam top-k. Metrik ini penting untuk mengetahui seberapa banyak dari seluruh kursus relevan yang dapat dijangkau oleh sistem rekomendasi.

$$
\text{Recall@k} = \frac{\text{Jumlah kursus relevan pada top-}k}{\text{Total jumlah kursus relevan}}
$$

<br>

Selain kedua metrik di atas, terdapat banyak metrik lainnya yang dapat digunakan seperti Mean Average Precision (MAP) dan Normalized Discounted Cumulative Gain (NDCG). Pemilihan metrik evaluasi dapat disesuaikan berdasarkan tujuan, kebutuhan, serta ketersediaan label relevansi pada dataset yang digunakan.

In [21]:
# Ground truth: kursus yang dianggap relevan untuk Senior Data Scientist
relevant_courses = {
    'Senior Data Scientist': {
        'Applied Data Science with Python',
        'Databases and SQL for Data Science with Python',
        'Python and Statistics for Financial Analysis',
        'Advanced Machine Learning',
        'Big Data Analysis'
    }
}

# Rekomendasi dari sistem
recommended_courses = {
    'Senior Data Scientist': [
        'Applied Data Science with Python',
        'Databases and SQL for Data Science with Python',
        'What is Data Science?',
        'Python and Statistics for Financial Analysis',
        'Generative Adversarial Networks (GANs)'
    ]
}

# Fungsi evaluasi

def precision_at_k(actual, predicted, k):
    predicted_at_k = predicted[:k]
    relevant = set(predicted_at_k).intersection(actual)
    return len(relevant) / k


def recall_at_k(actual, predicted, k):
    predicted_at_k = predicted[:k]
    relevant = set(predicted_at_k).intersection(actual)
    return len(relevant) / len(actual)


def f1_score_at_k(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def average_precision(actual, predicted, k):
    score = 0.0
    hits = 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual:
            hits += 1
            score += hits / (i + 1)
    return score / min(len(actual), k)

# Hitung metrik untuk k=5
k = 5
actual = relevant_courses['Senior Data Scientist']
predicted = recommended_courses['Senior Data Scientist']

precision = precision_at_k(actual, predicted, k)
recall = recall_at_k(actual, predicted, k)
f1 = f1_score_at_k(precision, recall)
avg_prec = average_precision(actual, predicted, k)

print(f"Precision@{k}: {precision:.2f}")
print(f"Recall@{k}: {recall:.2f}")
print(f"F1-Score@{k}: {f1:.2f}")
print(f"Mean Average Precision@{k}: {avg_prec:.2f}")

Precision@5: 0.60
Recall@5: 0.60
F1-Score@5: 0.60
Mean Average Precision@5: 0.55


Ini menunjukkan bahwa dari lima kursus teratas yang direkomendasikan sistem untuk satu pekerjaan, tiga di antaranya relevan dengan kebutuhan berdasarkan ground truth. Oleh karena itu, nilai Precision@5 dan Recall@5 sama-sama mencapai 0.60, yang berarti performa sistem cukup baik namun masih ada ruang untuk peningkatan.

Untuk mengetahui kualitas sistem rekomendasi secara keseluruhan, Precision@5 sebaiknya dihitung untuk semua pekerjaan yang ada dan dirata-ratakan (mean Precision@5). Hal ini akan memberikan gambaran umum mengenai performa sistem dalam skala besar.

Jika nilai precision@5 rendah, hal ini bisa menjadi indikator bahwa sistem perlu ditingkatkan, baik dari sisi algoritma yang digunakan, representasi fitur, strategi pemrosesan teks, atau metode pemetaan relevansi. Evaluasi kuantitatif seperti ini dapat menjadi dasar pengambilan keputusan dalam iterasi pengembangan sistem rekomendasi yang lebih akurat dan kontekstual.

# 5. Further Development

Berikut adalah beberapa ide untuk pengembangan lebih lanjut dari sistem rekomendasi ini:

1. Personalisasi berdasarkan profil penggunaSistem dapat menggabungkan informasi dari pengguna seperti pengalaman kerja, minat, latar belakang pendidikan, atau riwayat kursus sebelumnya untuk memberikan rekomendasi yang lebih sesuai secara individual. Hal ini memungkinkan pendekatan yang lebih adaptif dan relevan untuk setiap pengguna.

2. Pemrosesan Bahasa Alami (NLP) yang lebih mendalamDengan menerapkan model embedding seperti Word2Vec, BERT, atau SBERT, sistem dapat memahami konteks deskripsi pekerjaan dan kursus secara lebih akurat. Ini akan meningkatkan relevansi saat mencocokkan konten kursus dengan kebutuhan pekerjaan.

3. Pengayaan fitur kursus dan pekerjaanMenambahkan metadata seperti tingkat kesulitan, durasi, penyedia kursus, serta penilaian pengguna dapat memperkaya representasi masing-masing kursus. Dengan demikian, model dapat mempelajari lebih banyak dimensi dalam proses pencocokan.

4. Penggabungan pendekatan hybridSistem dapat memadukan content-based filtering dan collaborative filtering untuk memanfaatkan kekuatan keduanya. Ini akan membantu mengurangi masalah cold-start dan meningkatkan personalisasi.

5. Sistem umpan balik penggunaMenyediakan fitur untuk pengguna agar dapat memberi rating atau menandai rekomendasi sebagai tidak relevan. Data ini dapat digunakan untuk pelatihan ulang model dan evaluasi performa secara berkelanjutan.

6. Visualisasi dan monitoring performa sistemMenyediakan visualisasi seperti grafik Precision@K per pekerjaan, heatmap relevansi, atau tren performa dari waktu ke waktu akan membantu dalam pemantauan dan diagnosis sistem.

7. Penerapan algoritma ranking canggihMenggunakan model machine learning berbasis ranking seperti XGBoostRanker atau Learning-to-Rank untuk mengurutkan kursus berdasarkan skor relevansi. Hal ini memberikan kontrol lebih presisi terhadap urutan rekomendasi.

8. Evaluasi menyeluruh lintas pekerjaanMelakukan evaluasi pada seluruh jenis pekerjaan yang ada, bukan hanya satu contoh, untuk mengukur konsistensi dan generalisasi sistem. Hasilnya dapat dijadikan dasar perbaikan sistem secara menyeluruh.

9. Integrasi roadmap karierSistem dapat mempertimbangkan jalur karier pengguna dan merekomendasikan kursus yang mendukung transisi ke jenjang berikutnya, misalnya dari Data Analyst ke Data Scientist.

10. A/B Testing untuk sistem rekomendasiJika sistem digunakan dalam platform nyata, dapat dilakukan A/B testing untuk membandingkan versi algoritma yang berbeda dalam meningkatkan engagement pengguna.
